# This old notebook computes effeciencies with truth samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from exp_analysis_class import exp_analysis

In [3]:
case = 'heavy'

In [4]:
pars = {}
pars['heavy'] = {
    'm4' : [0.1, 0.12, 0.14],
    'mz_prime': [0.6, 1.25, 3],
}
pars['light'] = {
    'm4' : [0.1, 0.2, 0.4],
    'mz_prime': [0.03, 0.06, 0.09],
}

ctaus = [1, 5, 10, 20, 50, 100, 200]
ctaus = [0.1, 1, 10, 100, 1000]
!mkdir ../../fig/efficiency_vs_physics_pars2/
save_folder = "../../fig/efficiency_vs_physics_pars2/"

/bin/bash: port_used: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_port_used'
/bin/bash: wait_until_port_used: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_wait_until_port_used'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_module'
/bin/bash: find_port: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_find_port'
/bin/bash: random_number: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_random_number'
/bin/bash: helmod: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_helmod'
/bin/bash: source_helpers: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_source_h

In [20]:
#generation part
for m4, mz_prime in itertools.product(pars[case]['m4'], pars[case]['mz_prime']):
    print(m4, mz_prime)
    if case == 'light':
        dark_gen_run = f'cd ..; ./dark_gen.py --M4 {m4} --mzprime {mz_prime} --neval 100000 --noplot --hierarchy light_mediator'
    elif case == 'heavy':
        dark_gen_run = f'cd ..; ./dark_gen.py --M4 {m4} --mzprime {mz_prime} --UMU4 2.2e-7 --alpha_dark 0.4 --epsilon2 4.6e-4 --neval 100000 --noplot --hierarchy heavy_mediator'
    else:
        print('No good case selected')
    stream = os.popen(dark_gen_run)
    print(stream.read())

0.1 0.03
Model used 3+1
127.70051257843576

0.1 0.06
Model used 3+1
23.78856819533013

0.1 0.09
Model used 3+1
6.755615269969685

0.2 0.03
Model used 3+1
56.01908311955379

0.2 0.06
Model used 3+1
10.305762886618755

0.2 0.09
Model used 3+1
4.711280458351253

0.4 0.03
Model used 3+1
6.028369490229659

0.4 0.06
Model used 3+1
1.5944727491151698

0.4 0.09
Model used 3+1
0.6525314662899133



In [21]:
#efficiency plot
for m4 in pars[case]['m4']:
    for mz_prime in pars[case]['mz_prime']:
        aux_eff = []
        df = pd.read_pickle(f'../data/nd280_nu/3plus1/m4_{m4}_mzprime_{mz_prime}/MC_m4_{m4}_mzprime_{mz_prime}.pckl')
        initialise_df(df, ctaus)
        for ctau in ctaus:
            aux_eff.append(100*df['actual_weight', ''][df[f'total_selected_{ctau}', '']].sum()/df['actual_weight', ''].sum())
        plt.plot(ctaus, aux_eff, label=f"$m_Z'$ = {mz_prime*1000:.0f} MeV")
    plt.xlabel('ctau [cm]')
    plt.ylabel('efficiency [%]')
    # plt.xscale('log')
    plt.legend(loc='best', frameon=False)
    plt.title(f"{case} Z' case, $m_4$ = {m4*1000:.0f} MeV")
    plt.savefig(save_folder + f"{case}_m_4_{m4*1000:.0f}.png", dpi=250)
    plt.close()

In [10]:
# e+e- energy events histogram plot
# !mkdir ../../fig/e+e-deposited_energy/
this_save_folder = "../../fig/e+e-deposited_energy/"
for m4 in pars[case]['m4']:
    for mz_prime in pars[case]['mz_prime']:
        df = pd.read_pickle(f'../data/nd280_nu/3plus1/m4_{m4}_mzprime_{mz_prime}/MC_m4_{m4}_mzprime_{mz_prime}.pckl')
        initialise_df(df, ctaus)
        for ctau in ctaus:
            mask = df[f'total_selected_{ctau}', '']
            plt.hist(df['pee', 't'][mask],
                     bins=20, 
                     range=(0, 10),
                     weights=df['actual_weight', ''][mask],
                     histtype='step',
                     label=f"ctau = {ctau} cm, nevt = {df['actual_weight', ''][mask].sum():.2g}")
            # aux_eff.append(100*df['actual_weight', ''][df[f'total_selected_{ctau}', '']].sum()/df['actual_weight', ''].sum())
        # plt.plot(ctaus, aux_eff, label=f"$m_Z'$ = {mz_prime*1000:.0f} MeV")
        plt.xlabel('e=e- deposited energy [GeV]')
        plt.ylabel('Expected number of events')
        plt.ylim(bottom=0)
        # plt.xscale('log')
        plt.legend(loc='best', frameon=False)
        plt.title(f"{case} Z' case, $m_Z'$ = {mz_prime*1000:.0f}, $m_4$ = {m4*1000:.0f} MeV")
        plt.savefig(this_save_folder + f"{case}_m_z_prime_{mz_prime*1000:.0f}_m_4_{m4*1000:.0f}.png", dpi=250)
        plt.close()